In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
import re
from collections import Counter
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, q, k, v, mask=None):
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_weights = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_weights, v)

        return output, attn_weights

    def forward(self, q, k, v, mask=None):
        batch_size, seq_len = q.size(0), q.size(1)

        # Linear projections
        q = self.w_q(q).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        k = self.w_k(k).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        v = self.w_v(v).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        # Apply attention
        attn_output, attn_weights = self.scaled_dot_product_attention(q, k, v, mask)

        # Concatenate heads
        attn_output = attn_output.transpose(1, 2).contiguous().view(
            batch_size, seq_len, self.d_model
        )

        # Final linear projection
        output = self.w_o(attn_output)

        return output, attn_weights

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()

        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Multi-head attention with residual connection and layer norm
        attn_output, attn_weights = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Feed-forward with residual connection and layer norm
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout(ff_output))

        return x, attn_weights

class QATransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, num_heads=8, num_layers=4,
                 ff_dim=512, max_seq_len=512, dropout=0.1):
        super(QATransformer, self).__init__()

        self.d_model = d_model
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len

        # Embedding layers
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)

        # Transformer blocks
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # Output layer
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def create_mask(self, seq):
        """Create causal mask for decoder"""
        seq_len = seq.size(1)
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
        return mask.unsqueeze(0).unsqueeze(1)  # Add batch and head dimensions

    def forward(self, x, return_attentions=False):
        batch_size, seq_len = x.size()

        # Create causal mask
        mask = self.create_mask(x)

        # Embedding + positional encoding
        token_emb = self.token_embedding(x) * math.sqrt(self.d_model)
        x = self.positional_encoding(token_emb.transpose(0, 1)).transpose(0, 1)
        x = self.dropout(x)

        attentions = []

        # Pass through transformer blocks
        for transformer in self.transformer_blocks:
            x, attn_weights = transformer(x, mask)
            if return_attentions:
                attentions.append(attn_weights)

        # Output projection
        logits = self.output_layer(x)

        if return_attentions:
            return logits, attentions

        return logits

class SimpleTokenizer:
    def __init__(self):
        self.vocab = {}
        self.inverse_vocab = {}
        self.vocab_size = 0

        # Специальные токены
        self.special_tokens = {
            '<pad>': 0,
            '<unk>': 1,
            '<bos>': 2,
            '<eos>': 3,
            '<sep>': 4  # Разделитель вопроса и ответа
        }

        # Инициализация специальными токенами
        for token, idx in self.special_tokens.items():
            self.vocab[token] = idx
            self.inverse_vocab[idx] = token

        self.vocab_size = len(self.special_tokens)

    def build_vocab(self, texts, min_freq=1):
        """Построение словаря из текстов"""
        counter = Counter()

        for text in texts:
            # Улучшенная токенизация
            tokens = re.findall(r'\w+|[^\w\s]', text.lower())
            counter.update(tokens)

        # Добавляем частые слова в словарь
        for token, freq in counter.items():
            if freq >= min_freq and token not in self.vocab:
                self.vocab[token] = self.vocab_size
                self.inverse_vocab[self.vocab_size] = token
                self.vocab_size += 1

    def encode(self, text, max_length=None, add_special_tokens=True):
        """Кодирование текста в индексы"""
        tokens = re.findall(r'\w+|[^\w\s]', text.lower())

        if add_special_tokens:
            indices = [self.special_tokens['<bos>']]
        else:
            indices = []

        for token in tokens:
            if token in self.vocab:
                indices.append(self.vocab[token])
            else:
                indices.append(self.special_tokens['<unk>'])

        if add_special_tokens:
            indices.append(self.special_tokens['<eos>'])

        if max_length:
            if len(indices) < max_length:
                indices = indices + [self.special_tokens['<pad>']] * (max_length - len(indices))
            else:
                indices = indices[:max_length]

        return indices

    def decode(self, indices, skip_special_tokens=True):
        """Декодирование индексов в текст"""
        tokens = []
        for idx in indices:
            if idx in self.inverse_vocab:
                token = self.inverse_vocab[idx]
                if skip_special_tokens and token in self.special_tokens:
                    continue
                tokens.append(token)

        # Восстанавливаем текст с пробелами
        text = ' '.join(tokens)
        # Убираем лишние пробелы вокруг знаков препинания
        text = re.sub(r'\s+([^\w\s])', r'\1', text)
        text = re.sub(r'([^\w\s])\s+', r'\1', text)

        return text

class QADataset(torch.utils.data.Dataset):
    def __init__(self, questions, answers, tokenizer, max_length=128):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        # Форматируем: <bos> вопрос <sep> ответ <eos>
        input_text = f"{question} <sep> {answer}"

        # Токенизируем
        input_ids = self.tokenizer.encode(input_text, self.max_length)

        # Для языковой модели вход и цель одинаковы
        target_ids = input_ids.copy()

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'target_ids': torch.tensor(target_ids, dtype=torch.long)
        }

class QATrainer:
    def __init__(self, model, tokenizer, device=None):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def train(self, train_loader, val_loader=None, epochs=10, lr=0.001):
        optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=0.01)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        criterion = nn.CrossEntropyLoss(ignore_index=self.tokenizer.special_tokens['<pad>'])

        train_losses = []
        val_losses = []

        for epoch in range(epochs):
            # Training
            self.model.train()
            total_train_loss = 0

            for batch_idx, batch in enumerate(train_loader):
                input_ids = batch['input_ids'].to(self.device)
                target_ids = batch['target_ids'].to(self.device)

                optimizer.zero_grad()

                # Forward pass
                logits = self.model(input_ids)

                # Calculate loss - сдвигаем на 1 токен для языкового моделирования
                loss = criterion(
                    logits[:, :-1, :].contiguous().view(-1, self.model.vocab_size),
                    target_ids[:, 1:].contiguous().view(-1)
                )

                # Backward pass
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()

                total_train_loss += loss.item()

                if batch_idx % 10 == 0:
                    print(f'Epoch: {epoch+1}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

            scheduler.step()

            avg_train_loss = total_train_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation
            if val_loader:
                avg_val_loss = self.validate(val_loader, criterion)
                val_losses.append(avg_val_loss)
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

        return train_losses, val_losses

    def validate(self, val_loader, criterion):
        self.model.eval()
        total_val_loss = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(self.device)
                target_ids = batch['target_ids'].to(self.device)

                logits = self.model(input_ids)
                loss = criterion(
                    logits[:, :-1, :].contiguous().view(-1, self.model.vocab_size),
                    target_ids[:, 1:].contiguous().view(-1)
                )

                total_val_loss += loss.item()

        return total_val_loss / len(val_loader)

    def generate_answer(self, question, max_length=50, temperature=0.8):
        """Генерация ответа на вопрос"""
        self.model.eval()

        # Начинаем с вопроса и <sep>
        input_text = f"{question} <sep>"
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=False)

        # Добавляем <bos> в начало
        input_ids = [self.tokenizer.special_tokens['<bos>']] + input_ids
        generated = input_ids.copy()

        with torch.no_grad():
            for _ in range(max_length):
                # Подготавливаем вход
                input_tensor = torch.tensor([generated], dtype=torch.long).to(self.device)

                # Получаем логиты
                logits = self.model(input_tensor)

                # Берем логиты для последнего токена
                next_token_logits = logits[0, -1, :] / temperature

                # Применяем softmax для получения вероятностей
                probs = torch.softmax(next_token_logits, dim=-1)

                # Сэмплируем следующий токен
                next_token = torch.multinomial(probs, num_samples=1).item()

                # Добавляем токен к сгенерированной последовательности
                generated.append(next_token)

                # Если достигли <eos>, останавливаемся
                if next_token == self.tokenizer.special_tokens['<eos>']:
                    break

        # Извлекаем только ответ (всё что после <sep>)
        try:
            sep_idx = generated.index(self.tokenizer.special_tokens['<sep>'])
            answer_ids = generated[sep_idx + 1:]

            # Убираем <eos> если есть
            if self.tokenizer.special_tokens['<eos>'] in answer_ids:
                eos_idx = answer_ids.index(self.tokenizer.special_tokens['<eos>'])
                answer_ids = answer_ids[:eos_idx]
        except ValueError:
            # Если <sep> не найден, возвращаем всю сгенерированную последовательность
            answer_ids = generated[len(input_ids):]

        return self.tokenizer.decode(answer_ids)

def create_sample_data():
    """Создание примеров данных для обучения"""
    questions = [
        "какая столица франции",
        "как работает фотосинтез",
        "что такое искусственный интеллект",
        "как приготовить пасту",
        "какие планеты в солнечной системе",
        "что такое python",
        "как работает интернет",
        "что такое машинное обучение",
        "как сохранить здоровье",
        "что такое черная дыра"
    ]

    answers = [
        "столица франции париж",
        "фотосинтез преобразует свет в энергию",
        "ии это системы имитирующие человеческий интеллект",
        "варите пасту в кипящей воде 10 минут",
        "меркурий венера земля марс юпитер сатурн уран нептун",
        "python это язык программирования высокого уровня",
        "интернет это сеть соединенных компьютеров",
        "мо это алгоритмы обучающиеся на данных",
        "ешьте здоровую пищу и занимайтесь спортом",
        "черная дыра это область с огромной гравитацией"
    ]

    return questions, answers

def main():
    # Параметры
    D_MODEL = 756  # Уменьшим для более быстрого обучения
    NUM_HEADS = 4
    NUM_LAYERS = 6
    FF_DIM = 512
    MAX_LENGTH = 64
    BATCH_SIZE = 1
    EPOCHS = 100

    # Создание данных
    questions, answers = create_sample_data()

    # Инициализация токенизатора
    tokenizer = SimpleTokenizer()

    # Построение словаря
    all_texts = questions + answers
    tokenizer.build_vocab(all_texts, min_freq=1)

    print(f"Размер словаря: {tokenizer.vocab_size}")
    print("Пример словаря:", dict(list(tokenizer.vocab.items())[:10]))

    # Создание датасета и загрузчика
    dataset = QADataset(questions, answers, tokenizer, MAX_LENGTH)
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=BATCH_SIZE, shuffle=True
    )

    # Инициализация модели
    model = QATransformer(
        vocab_size=tokenizer.vocab_size,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        ff_dim=FF_DIM,
        max_seq_len=MAX_LENGTH
    )

    print(f"Модель создана. Параметров: {sum(p.numel() for p in model.parameters())}")

    # Обучение
    trainer = QATrainer(model, tokenizer)
    print("Начало обучения...")
    train_losses, _ = trainer.train(train_loader, epochs=EPOCHS, lr=0.001)

    # Тестирование
    test_questions = [
        "какая столица франции",
        "как работает фотосинтез",
        "что такое черная дыра"
    ]

    print("\n" + "="*50)
    print("ТЕСТИРОВАНИЕ МОДЕЛИ")
    print("="*50)

    for question in test_questions:
        answer = trainer.generate_answer(question, temperature=0.5)
        print(f"В: {question}")
        print(f"О: {answer}\n")

    # Сохранение модели
    torch.save({
        'model_state_dict': model.state_dict(),
        'tokenizer_vocab': tokenizer.vocab,
        'tokenizer_inverse_vocab': tokenizer.inverse_vocab,
        'model_config': {
            'vocab_size': tokenizer.vocab_size,
            'd_model': D_MODEL,
            'num_heads': NUM_HEADS,
            'num_layers': NUM_LAYERS,
            'ff_dim': FF_DIM,
            'max_seq_len': MAX_LENGTH
        }
    }, 'qa_model.pth')

    print("Модель сохранена в 'qa_model.pth'")

if __name__ == "__main__":
    main()

Размер словаря: 74
Пример словаря: {'<pad>': 0, '<unk>': 1, '<bos>': 2, '<eos>': 3, '<sep>': 4, 'какая': 5, 'столица': 6, 'франции': 7, 'как': 8, 'работает': 9}
Модель создана. Параметров: 18517586
Начало обучения...
Epoch: 1, Batch: 0, Loss: 4.5798
Epoch 1/100, Train Loss: 5.0639
Epoch: 2, Batch: 0, Loss: 4.4153
Epoch 2/100, Train Loss: 3.9020
Epoch: 3, Batch: 0, Loss: 2.8333
Epoch 3/100, Train Loss: 2.7407
Epoch: 4, Batch: 0, Loss: 2.0291
Epoch 4/100, Train Loss: 1.4041
Epoch: 5, Batch: 0, Loss: 1.8671
Epoch 5/100, Train Loss: 0.9608
Epoch: 6, Batch: 0, Loss: 0.5901
Epoch 6/100, Train Loss: 0.7506
Epoch: 7, Batch: 0, Loss: 0.6819
Epoch 7/100, Train Loss: 0.7282
Epoch: 8, Batch: 0, Loss: 0.4440
Epoch 8/100, Train Loss: 0.6386
Epoch: 9, Batch: 0, Loss: 0.5160
Epoch 9/100, Train Loss: 0.6394
Epoch: 10, Batch: 0, Loss: 0.4163
Epoch 10/100, Train Loss: 0.6126
Epoch: 11, Batch: 0, Loss: 0.8849
Epoch 11/100, Train Loss: 0.6012
Epoch: 12, Batch: 0, Loss: 0.5571
Epoch 12/100, Train Loss: 0.58